# New York AirBnB Exploratory data analysis  
An exercise developed for CodeOp Module 3 milestone  

- Author: Margherita Serena Ferrara  
- Date: 25/11/2023  
- Contact: margheritaserena.ferrara@gmail.com

[introduction]
- data source
- type of analysis performed
-- sections

[main findings]

[Set up]

In [ ]:
# Import revelant packages
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import folium
import os

In [ ]:
# Get folder paths
analysis_directory = os.getcwd()
parent_folder = os.path.dirname(analysis_directory)

## Section 1: Import data & clean data

[which files we import]

### Listings  
[dataset shoudl contain all listings in Milan]

#### Import dataset

In [ ]:
listings = pd.read_csv(parent_folder+'/MI AirBnB - dataset/listings.csv', delimiter = ',')

#### Explore data available
[identify varuiables of interest,
check whether variables need cleaning (NAs, weird symbols, convert in str/date etc)
]

In [ ]:
# Explore the data
listings.info()

##### Unique identifier  
[helpful to have unique identifier in teh dataset.
expert to be id]
we know there are 24177 entries

In [ ]:
listings['id'].nunique() == listings.shape[0]

#### Missing values
[description]

##### Drop fully missing variables
info() suggests that some variables are fully missing (such as neighbourhood). 
We drop these.

In [ ]:
listings = listings.dropna(axis=1, how='all')

##### Explore other variables that contain NAs

In [ ]:
listings.columns[listings.isna().any()]

One of the variables that contain NA values is 'bedrooms'
Explore this further

In [ ]:
listings['bedrooms'].value_counts(dropna=False)


5430 listings are registered with 0 bedrooms.
AirBnB website allows to select listings that have 1 or more bedrooms. 1 bedrooms listings also include studios.
> Decision: drop these observations

In [ ]:
#Store these observations in another dataset
listings_no_bedroom = listings[listings['bedrooms'].isna()]

listings = listings.dropna(subset=['bedrooms'])
# Check that observations have been dropped
listings[listings['bedrooms'].isna()]

Go back to list of NA variables

In [ ]:
listings.columns[listings.isna().any()]

they seem okay

#### Format variables
[description]

based on listings info notice two formatting to do:
price
calendar_last_scraped - not interested  

we explore price

In [ ]:
listings['price']

need to remove dollar sign and convert into float

In [ ]:
listings['price'] = listings['price'].str.replace('$', '')
# The variable price can't be immediately converted as float because the symbol ',' is not considered numerical.
listings['price'] = listings['price'].str.replace(',', '').astype(float)

# Note: quicker way to achieve this would have been with 
#   calendar['adjusted_price'] = calendar['adjusted_price'].replace('[$,]', '', regex=True).astype(float)

[conclusion: initial data cleaning of dataset listings is completed]

### Calendar

#### Import dataset

In [ ]:
calendar = pd.read_csv(parent_folder+'/MI AirBnB - dataset/calendar.csv')

#### Explore data available

In [ ]:
calendar.info()

In [ ]:
# Price, price_adjusted and date are identified as objects
calendar.head()

#### Unique identifiers
This case probably not
check that listing_id is not missing

In [ ]:
calendar[calendar['listing_id'].isna()]

No obs with missing id - good

In [ ]:
calendar['listing_id'].nunique()

#### Missing values

In [ ]:
columns_with_na = calendar.columns[calendar.isna().any()]

print(columns_with_na)

##### Drop fully missing variables

In [ ]:
calendar = calendar.dropna(axis=1, how='all')
calendar.info()

calendar.info() shows that no variable was fully missing.
However, some variables have na values(). we explore them

##### Explore other variables that contain NAs

look at how many observations are missing

In [ ]:
calendar[columns_with_na].isna().sum()

a relatively small amount of observations have nas.
Given taht we plan to merge this dataset with the listings one that contains these variables as well it's okay.

#### Format variables

price and date variables need formatting

In [ ]:
calendar['price'] = calendar['price'].replace('[$,]', '', regex=True).astype(float)
calendar['adjusted_price'] = calendar['adjusted_price'].replace('[$,]', '', regex=True).astype(float)
calendar['date'] = pd.to_datetime(calendar['date'], format = "%Y-%m-%d")
calendar.info()

calendar.info()shows that the conversion worked

[data cleaning done - for now]

## Section 2: Merge datasets

first rename variable in listings

In [ ]:
listings = listings.rename(columns={'id': 'listing_id'})

merge on variable listin_id

In [ ]:
mi_listings = pd.merge(listings, calendar, on='listing_id', how='outer', indicator=True)

how many observations were in common?
compare number of unique ids in the two datasets with those in the initial ones
mi_listings willhave same as listings
what about calendar?

In [ ]:
mi_listings['_merge'].value_counts()

keep only vars in common`

In [ ]:
# Extract calendar only data
right_only_data = mi_listings[mi_listings['_merge'] == 'right_only']['listing_id']

In [ ]:
mi_listings = mi_listings[mi_listings['_merge'] == 'both']

### Cross checks

The additional listings in calendar are those of the observations we dropped with 0 bedrooms?
we check this comparing listings_no_bedroom and calendar

In [ ]:
listings_no_bedroom = listings_no_bedroom.rename(columns={'id': 'listing_id'})
listings_no_bedroom_calendar = pd.merge(
    listings_no_bedroom['listing_id'], right_only_data,
    on = 'listing_id', how ='outer', indicator = True
    )

In [ ]:
listings_no_bedroom_calendar['_merge'].value_counts()

conclusion: ids dropped bc of no bedrooms
same things with code below:
[listings_2 = listings_2.rename(columns = {'id':'listing_id'})
merge = pd.merge(
    listings_2['listing_id'], calendar,
    on = 'listing_id', how ='outer', indicator = True
    )
    listings_2 = pd.read_csv(parent_folder+'/MI AirBnB - dataset/listings.csv', delimiter = ',')

    merge['_merge'].unique()
    
    ]

## Section 3: Exploratory data analysis

- first on separate datasets
- what are the min and max data?
- what is the distr of prices of listings?
- have they changed over time?

- price - histogram 
- price box plot - outliers?
 
- price over time
```
# Set the size of the figure
plt.figure(figsize=(20, 8))

sns.lineplot(
    data = calendar,
    x = "date",
    y = "price"
)
# Note: the line shows confidence interval and median
```  
- price per room type and number of beds
```
sns.boxplot(data = listings,
           y = 'price', 
           hue = 'room_type')

sns.boxplot(
    data = listings,
    x = 'bedrooms', 
    y = 'price')
```
- price and amenities (do charts in same figure)
```
# First, define the "subplot".
fig, axes = plt.subplots(
    nrows=2, 
    ncols=3,
    figsize=(20,6)
)

# Let's add a general title.
fig.suptitle("Three graphs of the Iris dataset")

# Add graphs at particular locations.
sns.histplot(
    data = data,
    x = "sepal_length",
    ax = axes[0,0]
    # Note: need to specify box row and column in ax to specify which chart is updated
)

```
-- possible charts for this


sns.boxplot(data = listings,
           y = 'price', 
           x = 'air_conditioning')
```
- distr of id in neighbourhoods - bar charts + geography  
- distr of price in neighbourhoods  

- price and n reviews and scores
```
listings[['price', 'number_of_reviews', 'review_scores_rating']].corr()
```